In [9]:

import pandas as pd

df = pd.read_csv('amazon_cells_labelled.csv')
df.columns=['Comment','Value 1','Value 2','Value 3','Value 4','Value 5']


,Comment,Value 1,Value 2,3,4,5
0,Good case,Excellent value.,1,NaN,NaN,NaN
1,Great for the jawbone.,1,NaN,NaN,NaN,NaN
2,Tied to charger for conversations lasting more...,0,NaN,NaN,NaN,NaN
3,The mic is great.,1,NaN,NaN,NaN,NaN
4,I have to jiggle the plug to get it to line up...,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
994,The screen does get smudged easily because it ...,0,NaN,NaN,NaN,NaN
995,What a piece of junk.. I lose more calls on th...,0,NaN,NaN,NaN,NaN
996,Item Does Not Match Picture.,0,NaN,NaN,NaN,NaN
997,The only thing that disappoint me is the infra...,0,NaN,NaN,NaN,NaN


In [19]:
X=df['Comment']
df['Value 1'] = pd.to_numeric(df['Value 1'], errors='coerce')

df = df[df['Value 1'].notnull() & (df['Value 1'] % 1 == 0)]

df.reset_index(drop=True, inplace=True)
y=df['Value 1']


<ipython-input-19-46d25553121e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Value 1'] = pd.to_numeric(df['Value 1'], errors='coerce')  # Convert to numeric, coerce errors to NaN


0      1.0
1      0.0
2      1.0
3      0.0
4      1.0
      ... 
765    0.0
766    0.0
767    0.0
768    0.0
769    0.0
Name: Value 1, Length: 770, dtype: float64

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2)
print(X_test)

266                    Rip off---- Over charge shipping.
224                                     Very good phone.
767    What a piece of junk.. I lose more calls on th...
553                                It was a great phone.
177    There's a horrible tick sound in the backgroun...
                             ...                        
384                                  I love this phone!.
702             much better than the hard plastic cases.
112    The majority of the Logitech earbud headsets f...
257    This is definitely a must have if your state d...
259                                  I love this device.
Name: Comment, Length: 154, dtype: object


In [27]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train = [str(text) for text in X_train]
X_test = [str(text) for text in X_test]


tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=128, return_tensors='tf')
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=128, return_tensors='tf')

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))


model = TFBertForSequenceClassification.from_pretrained("bert-base-cased", num_labels=1)  # num_labels=1 for binary classification

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metric = tf.keras.metrics.BinaryAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])


num_epochs = 1
history = model.fit(train_dataset.shuffle(1000).batch(8), epochs=num_epochs, validation_data=test_dataset.batch(8))


test_pred_probs = model.predict(test_dataset.batch(8))
logits = test_pred_probs.logits
test_preds = (tf.sigmoid(logits) > 0.5).numpy().astype(int)


accuracy = accuracy_score(y_test, test_preds)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


20/20 [==============================] - 33s 1s/step
Test Accuracy: 87.01%
